In [6]:
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [8]:
def build_df_predict(df_predict_cities, df_fit, z_predict):
    try:
        df_predict_cities["predict"] = z_predict.tolist()
    except:
        df_predict_cities["predict"] = 0
    df_predict_cities = df_predict_cities[["Estacao","predict"]]
    return pd.concat([df_predict_cities, df_fit])

In [9]:
def clean_df_fit(df_fit):
    estacao = df_fit["Estacao"]
    d_fit = {'Estacao': estacao, 'predict': df_fit["Precipitacao"]}
    return pd.DataFrame(data=d_fit)

In [10]:
def interp_precipitation(df_fit):
    fit_cities = df_fit["Estacao"].tolist()
    df_coordinates_cities = pd.read_csv('original_data/coordinates_cities.csv', delimiter=',', encoding='utf-8')
    df_predict_cities = df_coordinates_cities.loc[~df_coordinates_cities.Estacao.isin(fit_cities)]
    lon_fit = df_fit["long"]
    lat_fit = df_fit["lat"]
    if not(df_fit['Precipitacao'] == 0).all():
        z_fit = df_fit["Precipitacao"]
        OK = OrdinaryKriging(lon_fit, lat_fit, z_fit, variogram_model='linear', verbose=False,
                     enable_plotting=False, coordinates_type='geographic')
        lon_predict = df_predict_cities["long"]
        lat_predict = df_predict_cities["lat"]
        z_predict, ss_predict = OK.execute('points', lon_predict, lat_predict)
    else:
        z_predict = None
    df_fit_cleaned = clean_df_fit(df_fit)
    return build_df_predict(df_predict_cities, df_fit_cleaned, z_predict)

In [11]:
def get_df_interpolation_sum_monthy(df, year_month):
    df_year_month = df[df["concat"] == year_month]
    df_interp = interp_precipitation(df_year_month)
    df_interp["year"] = df_year_month["year"].unique()[0]
    df_interp["month"] = df_year_month["month"].unique()[0]
    df_interp["long"] = df_year_month["long"].unique()[0]
    df_interp["lat"] = df_year_month["lat"].unique()[0]
    df_interp["year-month"] = df_year_month["concat"].unique()[0]

    if (df_interp['predict'].min() < 0):
        difference = 1 + df_interp['predict'].min()
        scale = MinMaxScaler(feature_range=(0, df_interp['predict'].max() + difference ))
        df_interp["predict"] = scale.fit_transform(df_interp[['predict']].values.astype(float))
    df_interp.rename(columns={'predict':'predict_precipitation_normalized'}, inplace=True)
    return df_interp

In [12]:
def get_df_interpolation_day(df, day):
    df_day = df[df["Date"] == day]
    df_day_interp = interp_precipitation(df_day)
    df_day_interp["Data"] = df_day["Data"].unique()[0]
    df_day_interp["Date"] = df_day["Date"].unique()[0]
    df_day_interp["year"] = df_day["year"].unique()[0]
    df_day_interp["month"] = df_day["month"].unique()[0]
    df_day_interp["day"] = df_day["day"].unique()[0]
    df_day_interp["long"] = df_day["long"].unique()[0]
    df_day_interp["lat"] = df_day["lat"].unique()[0]
    if (df_day_interp['predict'].min() < 0):
        difference = 1 + df_day_interp['predict'].min()
        scale = MinMaxScaler(feature_range=(0, df_day_interp['predict'].max() + difference ))
        df_day_interp["predict"] = scale.fit_transform(df_day_interp[['predict']].values.astype(float))
    df_day_interp.rename(columns={'predict':'predict_precipitation_normalized'}, inplace=True)
    return df_day_interp

## Carregando dataframes filtrados

In [16]:
monthly_precipitation = pd.read_csv('filtered_data/result_monthly_precipitation_sum.csv', delimiter=',', encoding='utf-8')
days_precipitation = pd.read_csv('filtered_data/all_cities.csv', delimiter=',', encoding='utf-8')

## Criando coluna para facilitar no agrupamento de dados

In [20]:
monthly_precipitation['concat'] = monthly_precipitation["year"].map(str) + "-" + monthly_precipitation["month"].map(str)

## Interpolação da precipitação mensal de 2000-2017

In [22]:
interpolation_sum_monthy_list = list(map(lambda x: get_df_interpolation_sum_monthy(monthly_precipitation, x),
                                             monthly_precipitation["concat"].unique().tolist()))

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

## Salvando dataframe interpolado

In [ ]:
df_interpolation_sum_monthy = pd.concat(interpolation_sum_monthy_list)
df_interpolation_sum_monthy.to_csv("interpolation_data/interpolation_sum_monthy_2000_2017.csv", sep=',', encoding='utf-8')

## Pegando dataframe com dados de 2015 até 2017

In [25]:
days_2015_2017 = days_precipitation[days_precipitation["year"] >= 2015]

## Interpolação de dados diários de 2015 até 2017

In [26]:
interpolation_days_2015_2017_list = list(map(lambda x: get_df_interpolation_day(days_2015_2017, x), days_2015_2017["Date"].unique()))

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

...

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/airton/myenv/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

## Salvando dataframe interpolado 

In [27]:
df_interpolation_days_2015_2017 = pd.concat(interpolation_days_2015_2017_list)
df_interpolation_days_2015_2017.to_csv("interpolation_data/interpolation_days_2015_2017.csv", sep=',', encoding='utf-8')